In [186]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
search = "music" #hashtag, creator

URL = f"https://ads.tiktok.com/business/creativecenter/inspiration/popular/{search}/pc/en"
page = requests.get(URL)

soup = BeautifulSoup(page.text)
scripts = soup.find("script", id='__NEXT_DATA__')

In [30]:
search = "hashtag" #music
hashtag = "jhope"
period = "120"
country = "FR"

URL = f"https://ads.tiktok.com/business/creativecenter/{search}/{hashtag}/pc/en?countryCode={country}&period={period}"
page = requests.get(URL)

soup = BeautifulSoup(page.text)
script = soup.find('meta', content="width=device-width")

In [93]:
URL = f"https://ads.tiktok.com/business/creativecenter/trends/home/pc/en"
page = requests.get(URL)

soup = BeautifulSoup(page.text)

In [112]:
def trending_hashtag():
    result = []
    for s in soup.find_all('div', id='hashtagItemContainer'):
        tag = s.find('span', {"class": "titleText--qKHbP"}).text
        stats = s.find_all('span', {"class": "item-value--VAdnq"})
        posts = stats[0].text
        views = stats[1].text
        result.append([tag, posts, views])
    return result

In [114]:
def trending_music():
    result = []
    for s in soup.find_all('div', {"class": "music-name-wrap--FPaLL music-name-wrap--X6m9u"}):
        result.append(s.text)
    return result

In [124]:
def trending_creator():
    result = []
    for s in soup.find_all('div', id='creatorItemContainer'):
        tag = s.find('span', {"class": "music-name--LR+1s music-name--bEzZ1"}).text
        stats = s.find_all('span', {"class": "creator-data-value--CwFQt creator-data-value--S-K2V"})
        posts = stats[0].text
        views = stats[1].text
        result.append([tag, posts, views])
    return result

In [125]:
trending_creator()

[['MrBeast', '77M', '723M'],
 ['avani', '43M', '3B'],
 ['James Charles', '38M', '1B'],
 ['noah beck', '34M', '2B'],
 ['Kira Kosarin', '30M', '548M']]

In [126]:
soup

<!DOCTYPE html>
<html lang="en"><head><link href="/business/creativecenter/api/sitemap" rel="sitemap" title="Sitemap" type="application/xml"/><meta content="on" http-equiv="x-dns-prefetch-control"/><link as="font" href="https://sf16-scmcdn-sg.ibytedtos.com/obj/static-sg/ads/creative_hub_i18n/fonts/sofiapro-bold-webfont.woff2" rel="preload" type="font/woff2"/><link as="font" href="https://sf16-scmcdn-sg.ibytedtos.com/obj/static-sg/ads/creative_hub_i18n/fonts/sofiapro-medium-webfont.woff2" rel="preload" type="font/woff2"/><link as="font" href="https://sf16-scmcdn-sg.ibytedtos.com/obj/static-sg/ads/creative_hub_i18n/fonts/sofiapro-regular-webfont.woff2" rel="preload" type="font/woff2"/><link href="//sf16-scmcdn-sg.ibytedtos.com" rel="preconnect"/><link href="//p16-cc-sg.ibyteimg.com" rel="preconnect"/><link href="//p16-sign-sg.tiktokcdn.com" rel="preconnect"/><link href="//sf16-scmcdn-sg.ibytedtos.com" rel="dns-prefetch"/><link href="//p16-cc-sg.ibyteimg.com" rel="dns-prefetch"/><link hre

In [175]:
find_content = soup.find('script', id="__NEXT_DATA__" )
str_content = str(find_content).split('<')[1].split('>')[1]
json_content = json.loads(str_content)['props']['pageProps']['data']

In [178]:
json_content.keys()

dict_keys(['hashtags', 'tiktoks', 'trendMusic', 'creatorList'])

In [179]:
hashtags_content = json_content['hashtags']
tiktoks_content = json_content['tiktoks']
trendMusic_content = json_content['trendMusic']
creatorList_content = json_content['creatorList']

In [187]:
pd.DataFrame.from_records(tiktoks_content)

,cover,duration,id,region,title,countryCode,itemId,itemUrl
0,https://p16-sign.tiktokcdn-us.com/tos-useast5-...,14,7202044179273285675,United States,Fresh fried shell + hidden layer of cheese = C...,US,7192335448428137771,https://www.tiktok.com/@mnm_pipi/video/7192335...
1,https://p16-sign.tiktokcdn-us.com/tos-useast5-...,15,7202044179273220139,United States,Pick your $5 cravings trio on the app.,US,7191969275039599918,https://www.tiktok.com/@mnm_pipi/video/7191969...
2,https://p16-sign-sg.tiktokcdn.com/v0201/9bdea7...,15,7202044179273236523,United States,"Once you’ve met with a TurboTax expert, who’ll...",US,7197511320760077570,https://www.tiktok.com/@mnm_pipi/video/7197511...
3,https://p16-sign.tiktokcdn-us.com/tos-useast5-...,26,7202044179273252907,United States,"Want to get your share of a $500,000 prize poo...",US,7200469447600246062,https://www.tiktok.com/@mnm_pipi/video/7200469...
4,https://p16-sign.tiktokcdn-us.com/tos-useast5-...,10,7202044179273269291,United States,Would you wear these juicy shorts? #unexplaina...,US,7199770272159780138,https://www.tiktok.com/@mnm_pipi/video/7199770...


In [190]:
pd.DataFrame.from_records(creatorList_content)

,items,tcmId,userId,nickName,avatarUrl,countryCode,followerCnt,likedCnt,ttLink,tcmLink
0,"[{'vv': 46014935, 'itemId': '71444413226010575...",6921892282798964742,6614519312189947909,MrBeast,https://p16-sign-va.tiktokcdn.com/tos-maliva-a...,US,77008703,722899044,https://www.tiktok.com/@mrbeast,https://creatormarketplace.tiktok.com/ad#/auth...
1,"[{'vv': 25565120, 'itemId': '71274696206294254...",7186753996604833797,79609045006872576,avani,https://p16-sign-va.tiktokcdn.com/tos-maliva-a...,US,42937442,2946843590,https://www.tiktok.com/@avani,https://creatormarketplace.tiktok.com/ad#/auth...
2,"[{'vv': 8408213, 'itemId': '704503980991553873...",6797972504917835781,190068318278651904,James Charles,https://p16-sign-va.tiktokcdn.com/tos-maliva-a...,US,37574470,1099890752,https://www.tiktok.com/@jamescharles,https://creatormarketplace.tiktok.com/ad#/auth...
3,"[{'vv': 13209756, 'itemId': '71489267599903490...",6849865054498586629,6775033566016963589,noah beck,https://p16-sign-va.tiktokcdn.com/musically-ma...,US,34073518,2370862958,https://www.tiktok.com/@noahbeck,https://creatormarketplace.tiktok.com/ad#/auth...
4,"[{'vv': 4682345, 'itemId': '708578431047282206...",6715265773696188422,21210630,Kira Kosarin,https://p16-sign-va.tiktokcdn.com/tos-maliva-a...,US,29932294,547925235,https://www.tiktok.com/@kirakosarin,https://creatormarketplace.tiktok.com/ad#/auth...
